In [1]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product

### Problem Data

In [2]:
supply = [10000, 15000, 15000]
demand = [8000, 10000, 12000, 10000]
plants_num = len(supply)
whs_num = len(demand)
cartesian_plants_whs = list(product(range(plants_num), range(whs_num)))
transportation_cost = [0.60, 0.56, 0.22, 0.40, 0.36, 0.30, 0.28, 0.58, 0.65, 0.68, 0.55, 0.42]
arc_cost = {cartesian_plants_whs[i]:transportation_cost[i] for i in range(12)}
print(arc_cost)

{(0, 0): 0.6, (0, 1): 0.56, (0, 2): 0.22, (0, 3): 0.4, (1, 0): 0.36, (1, 1): 0.3, (1, 2): 0.28, (1, 3): 0.58, (2, 0): 0.65, (2, 1): 0.68, (2, 2): 0.55, (2, 3): 0.42}


### Decision Variable

In [3]:
model = gp.Model('Trasportation Network')

Restricted license - for non-production use only - expires 2025-11-24


In [4]:
number_product = model.addVars(cartesian_plants_whs, vtype=GRB.CONTINUOUS, lb=0.00, name='number of product sent')
print(number_product)

{(0, 0): <gurobi.Var *Awaiting Model Update*>, (0, 1): <gurobi.Var *Awaiting Model Update*>, (0, 2): <gurobi.Var *Awaiting Model Update*>, (0, 3): <gurobi.Var *Awaiting Model Update*>, (1, 0): <gurobi.Var *Awaiting Model Update*>, (1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (2, 0): <gurobi.Var *Awaiting Model Update*>, (2, 1): <gurobi.Var *Awaiting Model Update*>, (2, 2): <gurobi.Var *Awaiting Model Update*>, (2, 3): <gurobi.Var *Awaiting Model Update*>}


### Objective function

In [9]:
# model.setObjective(number_product.prod(transportation_cost))
model.setObjective(gp.quicksum(number_product[(i,j)]*arc_cost[(i,j)] for i in range(plants_num) for j in range(whs_num)))

### Constraints

In [10]:
# supply
supply_constr = model.addConstrs((gp.quicksum(number_product[(i,j)] for j in range(whs_num)) <= supply[i] for i in range(plants_num)), name='Supply')
# demand
demand_constr = model.addConstrs((gp.quicksum(number_product[(i,j)] for i in range(plants_num)) >= demand[j] for j in range(whs_num)), name='Demand')
print(supply_constr)
print(demand_constr)

{0: <gurobi.Constr *Awaiting Model Update*>, 1: <gurobi.Constr *Awaiting Model Update*>, 2: <gurobi.Constr *Awaiting Model Update*>}
{0: <gurobi.Constr *Awaiting Model Update*>, 1: <gurobi.Constr *Awaiting Model Update*>, 2: <gurobi.Constr *Awaiting Model Update*>, 3: <gurobi.Constr *Awaiting Model Update*>}


In [11]:
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 14 rows, 12 columns and 48 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 7e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+03, 2e+04]
LP warm-start: use basis
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4250000e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.425000000e+04


In [12]:
model.printAttr('X')


    Variable            X 
-------------------------
number of product sent[0,2]        10000 
number of product sent[1,0]         5000 
number of product sent[1,1]        10000 
number of product sent[2,0]         3000 
number of product sent[2,2]         2000 
number of product sent[2,3]        10000 
